In [ ]:
# now we model the multi-asset Geometric Brownian motion with Jump diffusion for two assets

import numpy as np

# multi-asset Jump diffusion model
class multiJump:
    
    def __init__(self, m = None, s0 = None, T = None, N = None, \
                 cov = None, prob1 = None, prob2 = None):
        self.m = m
        self.s0 = s0
        self.T = T
        self.N = N
        self.cov = cov
        self.prob1 = prob1
        self.prob2 = prob2
        
    # function that generates a specific process
    def gen_process(self):
        
        dt = np.divide(self.T,self.N)
        sigma = np.zeros(2)
        processes = np.zeros((self.N,2))
        processes[0,:] = np.log(self.s0)
        
        drift = np.zeros(2)
        diffusion = np.zeros(2)
        
        for i in range(1,self.N):
            z1 = np.random.normal(0,1)
            sigma[0] = cov[0,0]
            sigma[1] = cov[1,1]
            
            drift = 0.5*(sigma**2)*dt
            diffusion = np.sqrt(self.T/self.N)*np.matmul(np.linalg.cholesky(self.cov), \
                                               np.random.normal(0,1,size=2))
            jump = self.gen_jump(sigma[0], sigma[1])
            
            processes[i,:] = processes[i-1,:] - drift + diffusion + jump
        
        return np.exp(processes)
    
    # function that generates the jump part of the process
    def gen_jump(self, sigma1 = None, sigma2 = None):
    
        a=np.zeros(2)
        b=np.zeros(2)
        c=np.zeros(2)
        zeros=np.array((0,0))
        
        a[:]=(self.m*np.random.exponential()+np.sqrt(np.random.exponential())* \
              np.random.multivariate_normal(zeros,self.cov))
        b[:]=(sigma1*np.random.exponential()+np.sqrt(np.random.exponential())* \
              np.random.normal(0,sigma1),0)
        c[:]=(0,sigma2*np.random.exponential()+np.sqrt(np.random.exponential())* \
              np.random.normal(0,sigma2))
        
        z=np.ndarray(3,dtype=object)
        z[0]=a
        z[1]=b
        z[2]=c
        
        return np.random.choice(z,1,p=[1 - self.prob1 - self.prob2, self.prob1, self.prob2])
    
    # function that loops through the afordefined to generate multiple paths
    def gen_path(self, nbPaths = None):
        
        dt = np.divide(self.T,self.N)
        dualPaths = np.zeros(((self.N,2,nbPaths)))
        
        for i in range(nbPaths):
            dualPaths[:,:,i] = self.gen_process()
        
        return dualPaths
    
m = np.array([0.1,0.1])
s0 = np.array([100,100])
cov = np.array([[0.1, 0.05],[0.05, 0.2]])
    
sto = multiJump(m = m, s0 = s0, T = 1, N = 30, cov = cov, prob1 = 0.25, prob2 = 0.25)
spot = sto.gen_path(nbPaths = 1)
path = sto.gen_process()

print(path)